### Data Modelling for crime-house price prediction

In [1]:
## load dataset

import pandas as pd
# import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime
from scipy import stats
import seaborn as sns

crimes_2012 = pd.read_csv('Chicago_Crimes_2012_to_2017.csv', sep=',', error_bad_lines=False)
crimes_2012.drop_duplicates(subset=['ID', 'Case Number', 'Date'], inplace=True)
crimes_2012.drop(['Unnamed: 0','Case Number','IUCR','FBI Code','Updated On',
                  'X Coordinate','Y Coordinate'], inplace = True, axis = 1)
crimes_2012 = crimes_2012.dropna(axis = 0, how = 'any')

## date conversion
crimes_2012.Date = pd.to_datetime(crimes_2012.Date, format = '%m/%d/%Y %I:%M:%S %p')
# crimes_2012.index = pd.DatetimeIndex(crimes_2012.Date)
crimes_2012['Primary Type'] = pd.Categorical(crimes_2012['Primary Type'])
crimes_2012['Description'] = pd.Categorical(crimes_2012['Description'])
crimes_2012['Location Description'] = pd.Categorical(crimes_2012['Location Description'])

In [2]:
crimes_2012.head(5)

,ID,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,Year,Latitude,Longitude,Location
0,10508693,2016-05-03 23:40:00,013XX S SAWYER AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,10.0,24.0,29.0,2016,41.864073,-87.706819,"(41.864073157, -87.706818608)"
1,10508695,2016-05-03 21:40:00,061XX S DREXEL AVE,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,3.0,20.0,42.0,2016,41.782922,-87.604363,"(41.782921527, -87.60436317)"
2,10508697,2016-05-03 23:31:00,053XX W CHICAGO AVE,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,15.0,37.0,25.0,2016,41.894908,-87.758372,"(41.894908283, -87.758371958)"
3,10508698,2016-05-03 22:10:00,049XX W FULTON ST,BATTERY,SIMPLE,SIDEWALK,False,False,1532,15.0,28.0,25.0,2016,41.885687,-87.749516,"(41.885686845, -87.749515983)"
4,10508699,2016-05-03 22:00:00,003XX N LOTUS AVE,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,15.0,28.0,25.0,2016,41.886297,-87.761751,"(41.886297242, -87.761750709)"


In [3]:
### community area name
area_names = """
01	Rogers Park	
40	Washington Park
02	West Ridge	
41	Hyde Park
03	Uptown	
42	Woodlawn
04	Lincoln Square	
43	South Shore
05	North Center	
44	Chatham
06	Lakeview	
45	Avalon Park
07	Lincoln Park	
46	South Chicago
08	Near North Side	
47	Burnside
09	Edison Park	
48	Calumet Heights
10	Norwood Park	
49	Roseland
11	Jefferson Park	
50	Pullman
12	Forest Glen	
51	South Deering
13	North Park	
52	East Side
14	Albany Park	
53	West Pullman
15	Portage Park	
54	Riverdale
16	Irving Park	
55	Hegewisch
17	Dunning	
56	Garfield Ridge
18	Montclare	
57	Archer Heights
19	Belmont Cragin	
58	Brighton Park
20	Hermosa	
59	McKinley Park
21	Avondale	
60	Bridgeport
22	Logan Square	
61	New City
23	Humboldt Park	
62	West Elsdon
24	West Town	
63	Gage Park
25	Austin	
64	Clearing
26	West Garfield Park 	
65	West Lawn
27	East Garfield Park	
66	Chicago Lawn
28	Near West Side	
67	West Englewood
29	North Lawndale	
68	Englewood
30	South Lawndale	
69	Greater Grand Crossing
31	Lower West Side	
70	Ashburn
32	Loop	
71	Auburn Gresham	
33	Near South Side	
72	Beverly
34	Armour Square	
73	Washington Heights
35	Douglas	
74	Mount Greenwood
36	Oakland	
75	Morgan Park
37	Fuller Park	
76	O'Hare
38	Grand Boulevard	
77	Edgewater
39	Kenwood	
"""

In [8]:
## neighborhood : community name
neighborhood_Map = """
Cabrini–Green	Near North Side	
The Gold Coast	Near North Side	
Goose Island	Near North Side	
Magnificent Mile	Near North Side	
Old Town	Near North Side	
River North	Near North Side	
River West	Near North Side	
Streeterville	Near North Side	
Loop	Loop	
Near East Side	Loop	
South Loop	Loop	
West Loop Gate	Loop	
Dearborn Park	Near South Side	
Printer's Row	Near South Side	
South Loop	Near South Side	
Prairie Avenue Historic District	Near South Side	
Horner Park	North Center	
Roscoe Village	North Center	
Boystown	Lake View	
Lake View East	Lake View	
Graceland West	Lake View	
South East Ravenswood	Lake View	
Wrigleyville	Lake View	
Old Town Triangle	Lincoln Park	
Park West	Lincoln Park	
Ranch Triangle	Lincoln Park	
Sheffield Neighbors	Lincoln Park	
Wrightwood Neighbors	Lincoln Park	
Belmont Gardens	Avondale	
Chicago's Polish Village	Avondale	
Kosciuszko Park	Avondale	
Belmont Gardens	Logan Square	
Bucktown	Logan Square	
Kosciuszko Park	Logan Square	
Palmer Square	Logan Square	
East Rogers Park	Rogers Park	
Arcadia Terrace	West Ridge	
Peterson Park	West Ridge	
West Rogers Park	West Ridge	
Buena Park	Uptown	
Argyle Street	Uptown	
Margate Park	Uptown	
Sheridan Park	Uptown	
Ravenswood	Lincoln Square	
Ravenswood Gardens	Lincoln Square	
Rockwell Crossing	Lincoln Square	
Edison Park	Edison Park	
Big Oaks	Norwood Park	
Old Norwood Park	Norwood Park	
Oriole Park	Norwood Park	
Union Ridge	Norwood Park	
Gladstone Park	Jefferson Park	
Edgebrook	Forest Glen	
Old Edgebrook	Forest Glen	
South Edgebrook	Forest Glen	
Sauganash	Forest Glen	
Wildwood	Forest Glen	
Brynford Park	North Park	
Hollywood Park	North Park	
River's Edge	North Park	
Sauganash Woods	North Park	
Mayfair	Albany Park	
North Mayfair	Albany Park	
Ravenswood Manor	Albany Park	
Schorsch Forest View	O'Hare	
Andersonville	Edgewater	
Edgewater Beach	Edgewater	
Magnolia Glen	Edgewater	
Lakewood/Balmoral	Edgewater	
Belmont Central	Portage Park	
Władysławowo	Portage Park	
Six Corners	Portage Park	
Avondale Gardens	Irving Park	
Independence Park	Irving Park	
Kilbourn Park	Irving Park	
Little Cassubia	Irving Park	
Old Irving Park	Irving Park	
West Walker	Irving Park	
The Villa	Irving Park	
Belmont Heights	Dunning	
Belmont Terrace	Dunning	
Irving Woods	Dunning	
Schorsch Village	Dunning	
Montclare	Montclare	
Belmont Central	Belmont Cragin	
Hanson Park	Belmont Cragin	
Belmont Gardens	Hermosa	
Kelvyn Park	Hermosa	
East Village	West Town	
Noble Square	West Town	
Polish Downtown	West Town	
Pulaski Park	West Town	
Smith Park	West Town	
Ukrainian Village	West Town	
Wicker Park	West Town	
Galewood	Austin	
The Island	Austin	
West Garfield Park	West Garfield Park	
Fifth City	East Garfield Park	
Greektown	Near West Side	
Little Italy	Near West Side	
Tri-Taylor	Near West Side	
Lawndale	North Lawndale	
Homan Square	North Lawndale	
Douglas Park	North Lawndale	
Little Village	South Lawndale	
Heart of Chicago	Lower West Side	
Heart of Italy	Lower West Side	
Pilsen	Lower West Side	
East Pilsen	Lower West Side	
Chinatown	Armour Square	
Wentworth Gardens	Armour Square	
Bridgeport, Chicago	Armour Square	
Groveland Park	Douglas	
Lake Meadows	Douglas	
the Gap	Douglas	
Prairie Shores	Douglas	
South Commons	Douglas	
Oakland	Oakland	
Fuller Park	Fuller Park	
Bronzeville	Grand Boulevard	
Kenwood	Kenwood	
South Kenwood	Kenwood	
Washington Park	Washington Park	
East Hyde Park	Hyde Park	
Hyde Park	Hyde Park	
West Woodlawn	Woodlawn	
Jackson Park Highlands	South Shore	
Bridgeport	Bridgeport	
Grand Crossing	Greater Grand Crossing	
Parkway Gardens	Greater Grand Crossing	
Park Manor	Greater Grand Crossing	
LeClaire Courts	Garfield Ridge	
Sleepy Hollow	Garfield Ridge	
Vittum Park	Garfield Ridge	
Archer Heights	Archer Heights	
Brighton Park	Brighton Park	
McKinley Park 	McKinley Park	
Back of the Yards	New City	
Canaryville	New City	
West Elsdon	West Elsdon	
Gage Park	Gage Park	
Chrysler Village	Clearing	
Ford City	West Lawn	
West Lawn	West Lawn	
Lithuanian Plaza	Chicago Lawn	
Marquette Park	Chicago Lawn	
West Englewood	West Englewood	
Englewood	Englewood	
East Chatham	Chatham	
West Chatham	Chatham	
West Chesterfield	Chatham	
Avalon Park	Avalon Park	
Marynook	Avalon Park	
Stony Island Park	Avalon Park	
The Bush	South Chicago	
Burnside	Burnside	
Pill Hill	Calumet Heights	
Fernwood	Roseland	
Rosemoor	Roseland	
Cottage Grove Heights	Pullman	
London Towne	Pullman	
Jeffrey Manor	South Deering	
Trumbull Park	South Deering	
Altgeld Gardens	Riverdale	
Eden Green	Riverdale	
Golden Gate	Riverdale	
East Side	East Side	
West Pullman	West Pullman	
Hegewisch	Hegewisch	
Beverly View	Ashburn	
Mary Crest	Ashburn	
Parkview	Ashburn	
Scottsdale	Ashburn	
Wrightwood	Ashburn	
Auburn Gresham	Auburn Gresham	
Beverly	Beverly	
Brainerd	Washington Heights	
Longwood Manor	Washington Heights	
Princeton Park	Washington Heights	
Mount Greenwood Heights	Mount Greenwood	
Talley's Corner	Mount Greenwood	
Beverly Woods	Morgan Park	
Kennedy Park	Morgan Park	
West Morgan Park	Morgan Park	
"""

In [9]:
code_pairs = [[float(p[0]), p[1]] for p in [pair.strip().split('\t') for pair in area_names.strip().split('\n')]]
area_name_dic = {float(k[0]):k[1] for k in code_pairs} # code -> community area
area_toCode_dic = {k[1] : float(k[0]) for k in code_pairs} # community area -> code

code_pairs_neighborhoods = [[p[0], p[1]] for p in [pair.strip().split('\t') for pair in neighborhood_Map.strip().split('\n')]]
neighborhood_name_dic = {k[0]:k[1] for k in code_pairs_neighborhoods} #neighborhood -> community area

In [10]:
df=crimes_2012[['Primary Type', 'Location Description', 'Community Area']]
df['Community Name'] = df['Community Area'].map(area_name_dic)
df['neighbordhood'] = df['Community Name'].map(neighborhood_name_dic)
df.dropna().head()

C:\Users\jvret\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\jvret\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Primary Type,Location Description,Community Area,Community Name,neighbordhood
9,BATTERY,STREET,67.0,West Englewood,West Englewood
17,ROBBERY,SIDEWALK,40.0,Washington Park,Washington Park
19,BATTERY,APARTMENT,39.0,Kenwood,Kenwood
21,ASSAULT,OTHER,41.0,Hyde Park,Hyde Park
28,DECEPTIVE PRACTICE,RESIDENCE,58.0,Brighton Park,Brighton Park


### real eastate and geomap data

In [11]:
house_df = pd.read_excel('real_eastate_data_main.xlsx')
house_df.columns=house_df.columns.astype(str)
# house_df.columns=house_df.columns.astype(str)
house_df=house_df.set_index('Region')
house_df.columns=pd.to_datetime(house_df.columns)
house_ann= house_df.T.groupby(house_df.columns.year).agg('mean').T
# house_ann.reset_index(inplace=True)
house_ann.reset_index(inplace=True)
# df.reset_index(inplace=True) 
# house_ann =house_df.T.groupby(house_df.columns.year).agg(['mean','median']).T.unstack()
# house_ann.columns = house_ann.columns.map('{0[0]}{0[1]}'.format)
# s.columns=s.columns.map('_'.join)
# s.columns = s.columns.map('{0[0]}{0[1]}'.format)

In [12]:
regions = []
for r in house_df.index:
    if '-' in r:
        index = r.find('-')
        new_reg = r[index+2:]
        regions.append(new_reg)
    else:
        regions.append(r)

In [13]:
house_ann['area'] = regions
house_ann.head()

,Region,2012,2013,2014,2015,2016,2017,2018,2019,area
0,"Chicago, IL metro area",100.454545,108.383333,117.558333,123.616667,130.658333,137.991667,142.316667,137.15,"Chicago, IL metro area"
1,"Chicago, IL",125.400000,143.016667,161.600000,172.333333,183.283333,189.691667,192.891667,177.85,"Chicago, IL"
2,"Chicago, IL - Albany Park",91.727273,114.033333,134.991667,159.850000,165.791667,175.666667,178.425000,182.50,Albany Park
3,"Chicago, IL - Andersonville",186.100000,204.116667,217.925000,229.183333,248.016667,246.391667,253.733333,241.15,Andersonville
4,"Chicago, IL - Arcadia Terrace",75.490909,115.900000,138.283333,123.700000,148.900000,142.650000,168.691667,160.75,Arcadia Terrace


In [14]:
code = []
for row in house_ann.itertuples(index=True,name='Pandas'):
    #convert neighborhoods to community areas
    comm_area = None
    if row.area in neighborhood_name_dic.keys():
        comm_area = neighborhood_name_dic[row.area]
    else:
        comm_area = row.area
    
    if comm_area in area_toCode_dic.keys():
        code.append(area_toCode_dic[comm_area])
    else:
        code.append(None)

In [15]:
house_ann['Community Area'] = code
# house_ann.drop('Region', 1)
# house_ann.dropna().drop('Region', 1)
house_ann.head()

,Region,2012,2013,2014,2015,2016,2017,2018,2019,area,Community Area
0,"Chicago, IL metro area",100.454545,108.383333,117.558333,123.616667,130.658333,137.991667,142.316667,137.15,"Chicago, IL metro area",NaN
1,"Chicago, IL",125.400000,143.016667,161.600000,172.333333,183.283333,189.691667,192.891667,177.85,"Chicago, IL",NaN
2,"Chicago, IL - Albany Park",91.727273,114.033333,134.991667,159.850000,165.791667,175.666667,178.425000,182.50,Albany Park,14.0
3,"Chicago, IL - Andersonville",186.100000,204.116667,217.925000,229.183333,248.016667,246.391667,253.733333,241.15,Andersonville,77.0
4,"Chicago, IL - Arcadia Terrace",75.490909,115.900000,138.283333,123.700000,148.900000,142.650000,168.691667,160.75,Arcadia Terrace,2.0


In [16]:
# hola = pd.DataFrame(crimes_2012[crimes_2012['Primary Type'].isin(['THEFT','BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT'])]['Primary Type'])
# df.groupby(['Primary Type', 'Community Name'])df['Primary Type'].isin(['THEFT','BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT']).count()
# top_5_crime = pd.DataFrame(df[df['Primary Type'].isin(['THEFT','BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT'])]['Primary Type'])

crime_catg = df.groupby(['Community Name', 'Primary Type'])['Primary Type'].count().unstack()
crime_catg = crime_catg[['THEFT','BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT']]
crime_catg = crime_catg.dropna()
crime_catg.columns = crime_catg.columns.add_categories(['Community Name'])
crime_catg.columns = pd.Index(list(crime_catg.columns))
crime_catg.reset_index(inplace=True)
crime_catg.head(5)
# crime_catg.rename(columns=str).reset_index()

,Community Name,THEFT,BATTERY,CRIMINAL DAMAGE,NARCOTICS,ASSAULT
0,Albany Park,2782.0,2417.0,1825.0,666.0,736.0
1,Archer Heights,1295.0,746.0,809.0,250.0,277.0
2,Armour Square,1671.0,972.0,607.0,230.0,282.0
3,Ashburn,2769.0,1932.0,1864.0,608.0,880.0
4,Auburn Gresham,6467.0,8895.0,4455.0,4008.0,3051.0


In [18]:
### join crime data with real estate data
final_df = pd.merge(crime_catg, house_ann, left_on='Community Name', right_on='area')
final_df.drop(['Region', 'area', 'Community Area'], axis=1, inplace=True)
final_df.head(4)

,Community Name,THEFT,BATTERY,CRIMINAL DAMAGE,NARCOTICS,ASSAULT,2012,2013,2014,2015,2016,2017,2018,2019
0,Albany Park,2782.0,2417.0,1825.0,666.0,736.0,91.727273,114.033333,134.991667,159.850000,165.791667,175.666667,178.425000,182.50
1,Archer Heights,1295.0,746.0,809.0,250.0,277.0,91.954545,111.183333,112.866667,117.991667,119.541667,128.791667,146.058333,156.50
2,Ashburn,2769.0,1932.0,1864.0,608.0,880.0,86.954545,92.608333,102.333333,112.333333,121.966667,133.483333,145.916667,140.35
3,Auburn Gresham,6467.0,8895.0,4455.0,4008.0,3051.0,32.454545,30.900000,34.066667,40.391667,49.375000,52.025000,61.025000,57.20


### get annual crime count for each crime type for each region

In [28]:
crimes_2012['Date'] = pd.to_datetime(crimes_2012['Date'], format='%m/%d/%Y %H:%M:%S %p')
top_5_crimes = ['THEFT','BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT']
df = crimes_2012[crimes_2012['Primary Type'].isin(top_5_crimes)]
df_cross = (pd.crosstab(index=df['Community Area'],
                        columns=[df['Date'].dt.year, df['Primary Type']]).sort_index(axis=1, level=[1, 0]))
df_cross.columns = ['{} {}'.format(crime, year) for year, crime in df_cross.columns]
df_cross.reset_index(inplace=True)
df_cross.head()

,Community Area,ASSAULT 2012,ASSAULT 2013,ASSAULT 2014,ASSAULT 2015,ASSAULT 2016,BATTERY 2012,BATTERY 2013,BATTERY 2014,BATTERY 2015,...,NARCOTICS 2012,NARCOTICS 2013,NARCOTICS 2014,NARCOTICS 2015,NARCOTICS 2016,THEFT 2012,THEFT 2013,THEFT 2014,THEFT 2015,THEFT 2016
0,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,340,303,257,234,227,991,866,776,666,...,485,362,278,205,79,1043,1004,811,845,851
2,2.0,254,225,201,166,198,658,669,574,544,...,230,216,173,157,79,976,991,794,669,694
3,3.0,244,277,210,233,241,736,667,593,661,...,735,482,519,271,100,1338,1134,952,879,968
4,4.0,124,111,85,99,93,354,352,319,288,...,111,61,102,78,36,691,689,507,499,514


In [33]:
crimes_2012['Date'] = pd.to_datetime(crimes_2012['Date'], format='%m/%d/%Y %H:%M:%S %p')
top_5_crimes = ['THEFT','BATTERY', 'CRIMINAL DAMAGE', 'NARCOTICS', 'ASSAULT']
df = crimes_2012[crimes_2012['Primary Type'].isin(top_5_crimes)]


crime_cross = (pd.crosstab(index=df['Community Area'],
                           columns=[df['Date'].dt.year, df['Primary Type']]).sort_index(axis=1, level=[1, 0]))
crime_cross.columns = ['{} {}'.format(crime, year) for year, crime in crime_cross.columns]
crime_cross.reset_index(inplace=True)
crime_cross['Community Name'] = crime_cross['Community Area'].map(area_name_dic)
crime_cross.dropna().shape
crime_cross.head(5)

,Community Area,ASSAULT 2012,ASSAULT 2013,ASSAULT 2014,ASSAULT 2015,ASSAULT 2016,BATTERY 2012,BATTERY 2013,BATTERY 2014,BATTERY 2015,...,NARCOTICS 2013,NARCOTICS 2014,NARCOTICS 2015,NARCOTICS 2016,THEFT 2012,THEFT 2013,THEFT 2014,THEFT 2015,THEFT 2016,Community Name
0,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,1.0,340,303,257,234,227,991,866,776,666,...,362,278,205,79,1043,1004,811,845,851,Rogers Park
2,2.0,254,225,201,166,198,658,669,574,544,...,216,173,157,79,976,991,794,669,694,West Ridge
3,3.0,244,277,210,233,241,736,667,593,661,...,482,519,271,100,1338,1134,952,879,968,Uptown
4,4.0,124,111,85,99,93,354,352,319,288,...,61,102,78,36,691,689,507,499,514,Lincoln Square


In [36]:
### final table for crime data and real eastate data on one hot vector table
crime_finalDF = pd.merge(crime_cross, house_ann, left_on='Community Name', right_on='area')
# crime_finalDF.drop(['Region', 'area', 'Community Area'], inplace=True)
# crime_finalDF.head(10)
crime_finalDF.head(10)

,Community Area_x,ASSAULT 2012,ASSAULT 2013,ASSAULT 2014,ASSAULT 2015,ASSAULT 2016,BATTERY 2012,BATTERY 2013,BATTERY 2014,BATTERY 2015,...,2012,2013,2014,2015,2016,2017,2018,2019,area,Community Area_y
0,1.0,340,303,257,234,227,991,866,776,666,...,69.372727,94.916667,113.925000,135.400000,145.750000,149.733333,160.741667,164.50,Rogers Park,1.0
1,2.0,254,225,201,166,198,658,669,574,544,...,68.590909,84.816667,108.525000,123.741667,124.641667,128.966667,146.333333,154.20,West Ridge,2.0
2,3.0,244,277,210,233,241,736,667,593,661,...,153.690909,168.083333,180.841667,182.191667,199.158333,203.958333,213.233333,200.00,Uptown,3.0
3,4.0,124,111,85,99,93,354,352,319,288,...,174.390909,197.925000,227.450000,227.508333,231.283333,238.825000,251.316667,254.20,Lincoln Square,4.0
4,6.0,245,200,186,208,188,977,902,686,758,...,220.527273,239.575000,252.358333,262.575000,268.900000,281.583333,287.666667,278.55,Lakeview,6.0
5,7.0,135,108,116,78,105,396,387,369,332,...,266.890909,286.358333,311.000000,317.041667,321.358333,336.283333,342.775000,338.85,Lincoln Park,7.0
6,8.0,315,322,318,333,354,1332,1173,1174,1042,...,288.400000,324.225000,345.333333,350.825000,353.858333,365.808333,367.991667,362.85,Near North Side,8.0
7,9.0,14,11,11,18,26,64,56,34,48,...,186.536364,196.358333,228.283333,219.141667,211.533333,227.541667,229.933333,225.65,Edison Park,9.0
8,10.0,94,66,73,82,75,223,202,187,171,...,158.145455,181.541667,207.666667,203.391667,211.066667,218.083333,217.325000,211.25,Norwood Park,10.0
9,11.0,63,64,67,55,76,213,192,185,172,...,131.745455,148.900000,161.591667,169.525000,181.450000,189.158333,198.366667,195.45,Jefferson Park,11.0
